In [1]:
#best file already on Git

!pip install faker
!pip install uszipcode==0.2.6
from collections import Counter
import random
import numpy as np
from faker import Faker
import pandas as pd
import csv
import zipfile
import scipy.stats
import warnings
import string
import datetime

from string import ascii_lowercase
warnings.filterwarnings('ignore')
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)
fake = Faker()


     |████████████████████████████████| 1.5 MB 27.6 MB/s            
     |████████████████████████████████| 454 kB 24.2 MB/s            
     |████████████████████████████████| 120 kB 108.9 MB/s            
Start downloading data for rich info zipcode database, total size 450+MB ...
  10 MB finished ...
  20 MB finished ...
  30 MB finished ...
  40 MB finished ...
  50 MB finished ...
  60 MB finished ...
  70 MB finished ...
  80 MB finished ...
  90 MB finished ...
  100 MB finished ...
  110 MB finished ...
  120 MB finished ...
  130 MB finished ...
  140 MB finished ...
  150 MB finished ...
  160 MB finished ...
  170 MB finished ...
  180 MB finished ...
  190 MB finished ...
  200 MB finished ...
  210 MB finished ...
  220 MB finished ...
  230 MB finished ...
  240 MB finished ...
  250 MB finished ...
  260 MB finished ...
  270 MB finished ...
  280 MB finished ...
  290 MB finished ...
  300 MB finished ...
  310 MB finished ...
  320 MB finished ...
  330 MB finished ..

In [2]:
df_preauth = pd.read_csv('s3://{}/{}'.format('synthdata-dev/Pharmacy', "prior_auth_flag.csv"))
df_preauth=df_preauth.sort_values(by=['count'], ascending=False)
df_preauth = df_preauth.iloc[:1000]
df_preauth['ndccd'] = df_preauth['ndccd'].apply(lambda x: '{0:0>11}'.format(x))
preauth_lst = df_preauth["ndccd"].tolist()


In [4]:
fill_df = pd.read_csv('s3://{}/{}'.format('synthdata-dev/Pharmacy', "filling_provider_data.csv"))

pd.set_option('display.max_columns', None)
def fill_spec(x):
    fill_dict = {"1":["160"], '2':['159'],'3':["159", "160"],'4':["163"],'5':["161"],'6':["162"],'18':["164"],
                 '7':["165"],'14':["166"],'17':["167"],'11':["168"],'15':["169"],'9':["170"],'13':["171"],
                 '8':["172"],'10':["173"],'12':["174"],'16':["175"]}s
    
    if x in fill_dict.keys():
        return np.random.choice(fill_dict[x])
    return "176"

fill_df["primary_provider_type_code"] = fill_df["primary_provider_type_code"].astype(str)
fill_df["fill_spec"] = fill_df["primary_provider_type_code"].apply(lambda x: fill_spec(x))
#filler_specialty
fill_df["Filling"]=fill_df["nabp_number"].astype(str)+"%"+fill_df["legal_business_name"].astype(str)+"%"+fill_df["dba_name"].astype(str)+"%"+fill_df["doctor_name"].astype(str)+"%"+fill_df["store_number"].astype(str)+"%"+fill_df["address1"].astype(str)+"%"+fill_df["address2"].astype(str)+"%"+fill_df["city"].astype(str)+"%"+fill_df["state"].astype(str)+"%"+fill_df["zip"].astype(str)+"%"+fill_df["phone"].astype(str)+"%"+fill_df["extension"].astype(str)+"%"+fill_df["fax"].astype(str)+"%"+fill_df["email"].astype(str)+"%"+fill_df["cross_street_or_directions"].astype(str)+"%"+fill_df["county"].astype(str)+"%"+fill_df["msa"].astype(str)+"%"+fill_df["pmsa"].astype(str)+"%"+fill_df["_24_hour_operation"].astype(str)+"%"+fill_df["provider_hours"].astype(str)+"%"+fill_df["congressional_voting_district"].astype(str)+"%"+fill_df["language_code_1"].astype(str)+"%"+fill_df["language_code_2"].astype(str)+"%"+fill_df["language_code_3"].astype(str)+"%"+fill_df["language_code_4"].astype(str)+"%"+fill_df["language_code_5"].astype(str)+"%"+fill_df["store_open_date"].astype(str)+"%"+fill_df["store_closure_date"].astype(str)+"%"+fill_df["mailing_address1"].astype(str)+"%"+fill_df["mailing_address2"].astype(str)+"%"+fill_df["mailing_address_city"].astype(str)+"%"+fill_df["mailing_address_state"].astype(str)+"%"+fill_df["mailing_address_zip"].astype(str)+"%"+fill_df["contact_last_name"].astype(str)+"%"+fill_df["contact_first_name"].astype(str)+"%"+fill_df["contact_middle_initial"].astype(str)+"%"+fill_df["contact_title"].astype(str)+"%"+fill_df["contact_phone"].astype(str)+"%"+fill_df["contact_extension"].astype(str)+"%"+fill_df["contact_email"].astype(str)+"%"+fill_df["dispenser_class_code"].astype(str)+"%"+fill_df["primary_provider_type_code"].astype(str)+"%"+fill_df["secondary_provider_type_code"].astype(str)+"%"+fill_df["tertiary_provider_type_code"].astype(str)+"%"+fill_df["medicare_provider_id"].astype(str)+"%"+fill_df["npi"].astype(str)+"%"+fill_df["dea_registration_number"].astype(str)+"%"+fill_df["dea_expiration_date"].astype(str)+"%"+fill_df["tax_id"].astype(str)+"%"+fill_df["state_income_tax_id"].astype(str)+"%"+fill_df["deactivation_code"].astype(str)+"%"+fill_df["reinstatement_code"].astype(str)+"%"+fill_df["reinstatement_date"].astype(str)+"%"+fill_df["fill_spec"].astype(str)
fill_lst = fill_df["Filling"].tolist()

fill_df = fill_df.rename(columns = {"address1":"pharmacy_address_line1", "address2": "pharmacy_address_line2", "city":"pharmacy_city", "state":"pharmacy_state", "phone":"pharmacy_phone_number", "extension":"pharmacy_phone_number_extension",
                                    "fax":"pharmacy_fax","email":"pharmacy_email", "contact_last_name":"filling_provider_lname", "contact_first_name":"filling_provider_fname", "npi":"filling_provider_npi", "tax_id":"filling_provider_taxid", 
                                    "medicare_provider_id":"filling_provider_custid","nabp_number":"pharmacy_nabp", "legal_business_name":"pharmacy_affiliation", "store_number":"network_number", "dispenser_class_code":"dispenser_type",
                                    "primary_provider_type_code":"dispenser_type_code","cross_street_or_directions":"pharmacy_cross_street_or_directions", "county":"pharmacy_county", "msa":"pharmacy_msa", "pmsa":"pharmacy_pmsa", "_24_hour_operation":"pharmacy_24_hour_operation",
                                    "provider_hours":"pharmacy_provider_hours", "congressional_voting_district":"pharmacy_congressional_voting_district", "language_code_1":"pharmacy_language_code_1", "language_code_2":"pharmacy_language_code_2",
                                    "language_code_3":"pharmacy_language_code_3", "language_code_4":"pharmacy_language_code_4", "language_code_5":"pharmacy_language_code_5", "store_open_date":"pharmacy_store_open_date", "store_closure_date":"pharmacy_store_closure_date",
                                    "mailing_address1":"pharmacy_mailing_address1", "mailing_address2":"pharmacy_mailing_address2", "mailing_address_city":"pharmacy_mailing_address_city", "mailing_address_state":"pharmacy_mailing_address_state",
                                    "mailing_address_zip":"pharmacy_mailing_address_zip", "contact_middle_initial":"pharmacy_contact_middle_initial", "contact_title":"pharmacy_contact_title", "contact_phone":"pharmacy_contact_phone",
                                    "contact_extension":"pharmacy_contact_extension", "contact_email":"pharmacy_contact_email", "secondary_provider_type_code":"filling_secondary_provider_type_code", "tertiary_provider_type_code":"filling_tertiary_provider_type_code", 
                                    "dea_registration_number":"filling_dea_registration_number", "dea_expiration_date":"filling_dea_expiration_date", "state_income_tax_id":"filling_state_income_tax_id", "deactivation_code":"filling_deactivation_code", 
                                    "reinstatement_code":"filling_reinstatement_code", "reinstatement_date":"filling_reinstatement_date", "dba_name":"pharmacy_dba_name", "fill_spec":"filler_specialty"})#, inplace=True)

fill_df = fill_df.drop(columns=['Filling'])
fill_col = list(fill_df.columns)



,pharmacy_nabp,pharmacy_affiliation,pharmacy_dba_name,doctor_name,network_number,pharmacy_address_line1,pharmacy_address_line2,pharmacy_city,pharmacy_state,zip,pharmacy_phone_number,pharmacy_phone_number_extension,pharmacy_fax,pharmacy_email,pharmacy_cross_street_or_directions,pharmacy_county,pharmacy_msa,pharmacy_pmsa,pharmacy_24_hour_operation,pharmacy_provider_hours,pharmacy_congressional_voting_district,pharmacy_language_code_1,pharmacy_language_code_2,pharmacy_language_code_3,pharmacy_language_code_4,pharmacy_language_code_5,pharmacy_store_open_date,pharmacy_store_closure_date,pharmacy_mailing_address1,pharmacy_mailing_address2,pharmacy_mailing_address_city,pharmacy_mailing_address_state,pharmacy_mailing_address_zip,filling_provider_lname,filling_provider_fname,pharmacy_contact_middle_initial,pharmacy_contact_title,pharmacy_contact_phone,pharmacy_contact_extension,pharmacy_contact_email,dispenser_type,dispenser_type_code,filling_secondary_provider_type_code,filling_tertiary_provider_type_code,filling_provider_custid,filling_provider_npi,filling_dea_registration_number,filling_dea_expiration_date,filling_provider_taxid,filling_state_income_tax_id,filling_deactivation_code,filling_reinstatement_code,filling_reinstatement_date,filler_specialty
0,5658236,ALBERTSONS LLC,SAV-ON PHARMACY #3171,NaN,3171,1500 N H ST,NaN,LOMPOC,CA,934362812,8057370416,NaN,8057403017,MANAGEDCARE.ENROLLMENTS@ALBERTSONS.COM,NaN,6083,7480,NaN,N,11017209203092040920509206092070917,24,en,NaN,NaN,NaN,NaN,3022016,0,250 E PARKCENTER BLVD,MAILSTOP SWC2-D,BOISE,ID,837063940,GIANNAKOPOULOS,KATHY,NaN,"MANAGER, 3RD PARTY IMPLEMENTAT",2083953954,NaN,MANAGEDCARE.ENROLLMENTS@ALBERTSONS.COM,2,1,13,NaN,0483391277,1811352313,FA5836145,6302022,820184434,NaN,NaN,NaN,0,160
1,118807,MR DISCOUNT DRUGS OF BUTLER INC ...,MR DISCOUNT DRUGS OF BUTLER ...,...,,604 EAST PUSHMATAHA ST ...,...,BUTLER,AL,36904,2054592990,,2054593926,MRDISCOUNT@TDS.NET ...,...,1023,,,N,1 23318333184331853318633187,07,en,,,,,8011983,0,604 EAST PUSHMATAHA ST ...,...,BUTLER,AL,36904,MANLEY,GREGORY,K,"OWNER, PIC, AO",2054592990,,MRDISCOUNT@TDS.NET ...,1,1,,,0840750001,1831188010,AM2766446,1312024,630878665,1201894,,,0,160
2,118819,"WAL-MART STORES EAST, LP ...",WALMART PHARMACY 10-0766 ...,...,100766,3100 HOUGH RD ...,...,FLORENCE,AL,35630,2567674725,,,ENROLLM@WAL-MART.COM ...,...,1077,2650,,N,11018209213092140921509216092170919,05,en,,,,,0,0,702 SW 8TH ST. ...,...,BENTONVILLE,AR,727160445,LITTLE,SARAH,,"DIRECTOR, HEALTHCARE CONTRACTI",4792772500,,ENROLLM@WAL-MART.COM ...,2,1,13,,4355050547,1992722276,BW5383548,5312022,710862119,,,,0,160
3,118821,"WAL-MART STORES EAST, LP ...",WALMART PHARMACY 10-0660 ...,...,100660,301 AVALON AVE ...,...,MUSCLE SHOALS,AL,356612811,2563867384,,2563867386,ENROLLM@WAL-MART.COM ...,...,1033,2650,,N,11018209213092140921509216092170919,05,,,,,,7041981,0,702 SW 8TH ST. ...,...,BENTONVILLE,AR,727160445,LITTLE,SARAH,,"DIRECTOR, HEALTHCARE CONTRACTI",4792772500,,ENROLLM@WAL-MART.COM ...,2,1,13,,4355051524,1801813183,BW5383233,5312022,710862119,,,,0,160
4,118883,"WAL-MART STORES EAST, LP ...",WALMART PHARMACY 10-0762 ...,...,100762,9248 PARKWAY EAST ...,...,BIRMINGHAM,AL,35206,2058330524,,,ENROLLM@WAL-MART.COM ...,...,1073,1000,,N,11018209213092140921509216092170919,07,en,,,,,0,0,702 SW 8TH ST. ...,...,BENTONVILLE,AR,727160445,LITTLE,SARAH,,"DIRECTOR, HEALTHCARE CONTRACTI",4792772500,,ENROLLM@WAL-MART.COM ...,2,1,13,,4355051601,1710904099,BW5383500,5312022,710862119,,,,0,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49520,117502,ALABAMA CVS PHARMACY LLC ...,CVS PHARMACY #04859 ...,...,04859,7501 FIRST AVE N ...,...,BIRMINGHAM,AL,35206,2058332972,,4017351080,ENROLLMENT_RETAIL@CVSHEALTH.COM ...,...,1073,1000,,N,11018209213092140921509216092170918,07,,,,,,11241996,0,1 CVS DRIVE ...,BOX 1075 ...,WOONSOCKET,RI

In [6]:
def syn_data(taxonomy, folder, prov_pct):
    num_claims_csv ="Input/Pharmacy/"+ folder+"/num_claim_df.csv"
    all_combo_csv = "Input/Pharmacy/"+ folder+"/all_combinations_age.csv"
    location_csv = "Input/Pharmacy/"+ folder+"/location_data.csv"
    c_amt_csv = "Input/Pharmacy/"+ folder+"/c.csv"
    d_amt_csv ="Input/Pharmacy/"+  folder+"/d.csv"
    refill_csv = "Input/Pharmacy/"+  folder+"/refills.csv"
    
    nopay_df = pd.read_csv("Input/Pharmacy/"+ folder+"/hdr_pay.csv")
    nopay_df = nopay_df.sort_values(by=['count'], ascending=False)
    nopay_df["ndccd"] = nopay_df["ndccd"].astype(str)
    nopay_df['ndccd'] = nopay_df['ndccd'].apply(lambda x: '{0:0>11}'.format(x))
    nopay_lst = nopay_df["ndccd"].tolist()
    
    def zcode(x):
        return search.by_zipcode(x).city
    
    def spliting(name, delineator = "@"):
        final_df[name] = final_df[name].apply(lambda x: x.split(delineator))
    
    def preauth(x):
        if x in preauth_lst:
            return "Y"
        return "N"
    def spliting3(col_name, num):
        col_name = col_name.split('@')
        null_vals = num - len(col_name)
        null_lst = [""] * null_vals
        col_name = col_name + null_lst
        return col_name        
    
    def in_network(x):
        if x =="Par":
            return "N"
        return "Y"
        
    rendering_df = pd.DataFrame()
    num_claims_df = pd.read_csv(num_claims_csv)
    num_providers = num_claims_df.shape[0]//prov_pct#may want to edit this, going lower than about 10 requires more ram
    rendering_df["prescriber_specialty"] = str(taxonomy)
    rendering_df["carrier_id"]= np.random.randint(10000000,99999999, size=num_providers).astype(str)
    rendering_df['prescribing_provider_name'] = [fake.last_name() for i in range(num_providers)]
    rendering_df['prescriber_phone_number'] = np.random.randint(1000000000,9999999999, size=num_providers).astype(str)
    rendering_df['1'] = [fake.building_number() for i in range(num_providers)]
    rendering_df["2"] = [fake.street_name() for i in range(num_providers)]
    rendering_df['prescriber_address_line1'] = rendering_df["1"]+" "+rendering_df["2"]
    rendering_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], num_providers, p=[0.05, 0.15, 0.8])
    rendering_df["2"] = [fake.building_number() for i in range(num_providers)]
    rendering_df["prescriber_address_line2"] = rendering_df["1"]+" "+rendering_df["2"]
    rendering_df.loc[rendering_df['prescriber_address_line2'].str.contains('Null', case=False), 'prescriber_address_line2'] = ''
    
    rendering_df["prescribing_provider_custid"] = np.random.randint(1000000,99999999, size=num_providers).astype(str)
    rendering_df["prescribing_provider_taxid"]= np.random.randint(100000000,999999999, size=num_providers).astype(str)
    rendering_df['1'] = ''.join(random.choice(ascii_lowercase) for x in range(2))
    
    rendering_df["prescriber_dea_number"] = np.random.randint(100000,900000, size=num_providers).astype(str)
    rendering_df["prescriber_dea_number"] = rendering_df['1']+"-"+rendering_df["prescriber_dea_number"]
    rendering_df["prescribing_provider_npi"] = np.random.randint(1000000000,9999999999, size=num_providers)
    rendering_df["prescribing_provider_npi"] = rendering_df["prescribing_provider_npi"].astype(str)
    rendering_df['ren_id'] = range(1, 1+len(rendering_df))
    rendering_df = rendering_df.drop(columns=['1',"2"])
    
    rend_file = pd.DataFrame()
    rend_file["ProviderNPI"]=rendering_df["prescribing_provider_npi"] 
    rend_file["predicted_specialty_id_1"] = taxonomy
    rend_file["number_of_predicted_specialties"]="1"
    rend_prov_null_lst = ["etl_provider_key", "provider_npi", "specialty_1_probability_value", "predicted_specialty_id_2", "specialty_2_probability_value", "predicted_specialty_id_3", "specialty_3_probability_value", "customer_provided_specialty_desc",
                          "npi_reg_primary_taxonomy_code", "used_cust_specialty_flg"]
    rend_file[rend_prov_null_lst]=""
    rend_file.to_csv("Out/Pharmacy/"+str(folder)+"rend_file.csv",  index=False)
    prov_file = pd.DataFrame(index=np.arange(num_providers))
    prov_null_lst = ["cdx_claim_id", "claim_specialty_id", 'claim_specialty_probability_value', "cust_claim_specialty_desc", "etl_provider_key", "provider_npi"]
    prov_file[prov_null_lst]=""
    prov_file.to_csv("Out/Pharmacy/"+str(folder)+"prov_file.csv",  index=False)
    
    #How many providers do we want to synthesize?
    ## User defined for each specialty
    num_iter = 100000
    num_claims_avg_lst = []
    
    arr_claims = num_claims_df.num_claims.values
    n_C = round(0.1*len(num_claims_df))
    for ii in range(0, num_iter):
        num_claims_avg_lst.append(round(arr_claims[np.random.choice(arr_claims.shape[0], n_C)].mean()))
    
    dist_mean = np.round(np.mean(num_claims_avg_lst))
    dist_std = np.round(np.std(num_claims_avg_lst))
    group = [int(round(x)) for x in np.random.normal(loc=dist_mean, scale=3*dist_std,size=num_providers)]
    scipy.stats.describe(group)
    
    ### Determine probability of different combinations 
    ### We'll eventually bring inference (of number of claims for each combination) from all customers for each specialty.
    all_combinations = pd.read_csv(all_combo_csv)
    
    all_combinations= all_combinations[all_combinations["combos"].str.contains("MA")==False]
    all_combinations= all_combinations[all_combinations["combos"].str.contains("9999")==False]
    all_combinations= all_combinations[all_combinations["combos"].str.contains("D7")==False]

    print("!"+ str(datetime.datetime.now()))
    
    t1_df = pd.read_csv(c_amt_csv)
    t2_df = pd.read_csv(d_amt_csv)
    
    t1_lst = t1_df['combo'].tolist()
    t2_lst = t2_df['combo'].tolist()
    t3_lst = t1_lst and t2_lst
    
    all_combinations["combos1"] = all_combinations["combos"].apply(lambda x: x.split(":"))
    safe_lst = []
    all_combinations['combos1'] = all_combinations['combos'].str.replace(';', '@')
    all_combinations["combos1"] = all_combinations["combos1"].apply(lambda x: x.split(":"))
    all_combinations['test1'] = all_combinations['combos1'].apply(lambda x: Counter(x))
    test2 = Counter(t3_lst)
    all_combinations['safe'] = all_combinations['test1'].apply(lambda x: True if sum(x.values()) == sum((x & test2).values()) else False)    
    
    sel_combinations = all_combinations.loc[all_combinations['safe'] == True]
    sel_combinations =sel_combinations.drop(columns=["combos1","safe","test1"])
    refill_df = pd.read_csv(refill_csv)
    refill_df = refill_df.drop(columns=['Unnamed: 0'])
    
    sel_combinations = sel_combinations.merge(refill_df, how='left', on='combos')
    #sel_combinations["prob_claims"] = sel_combinations["prob_claims"]*sel_combinations["prob_size"].astype(float)
    
    ## Unique combinations and corresponding weights
    comb_list = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).values
    weights = sel_combinations['prob_claims'].tolist()
    
    sel_columns= sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns
    print(num_providers)
    cdf = [weights[0]]
    for w in weights[1:]:
        cdf.append(cdf[-1]+w)
   
    final_df_list = []
    random_outliers = random.sample(range(0, num_providers), int(round(0.001*num_providers)))
    for ii in range(0, num_providers):
        if ii in random_outliers:
            tmp = pd.DataFrame(random.choices(comb_list, k=group[ii]), 
                           columns = sel_columns)
        else:
            tmp = pd.DataFrame(random.choices(comb_list, cum_weights = cdf, k=group[ii]), 
                           columns = sel_columns)
        tmp['ren_id'] = str(ii+1)
        #tmp['prov_claim_id'] = ['claim_' + str(x) for x in range(0, group[ii])]
        final_df_list.append(tmp)
    final_df = pd.concat(final_df_list)
    print('^'+str(datetime.datetime.now()))
    final_df["combos"] = final_df["combos"].apply(lambda x: x.split(":"))
    
    zip_length = rendering_df.shape[0] + 10
    zip_df =pd.read_csv(location_csv)
    zip_combinations = zip_df
    comb_list = zip_combinations.drop(['prob_combo'], axis = 1).values
    weights = zip_combinations['prob_combo'].tolist()
    
    zip_df = pd.DataFrame(random.choices(comb_list, weights, k=zip_length), columns = zip_df.drop(['prob_combo'], axis = 1).columns)
    rend_zips = zip_df['prescribing_provider_zipcode'].tolist()
    rendering_df['prescribing_provider_zipcode'] = np.random.choice(rend_zips, rendering_df.shape[0])
    rend_zip_df = rendering_df.merge(zip_df.drop_duplicates(), on='prescribing_provider_zipcode')
    dfRendChoices = pd.DataFrame()
    rend_tin = rend_zip_df['prescribing_provider_npi'].unique()
    i = 0
    for tin in rend_tin:
        if i%10 == 0:
            print(i)
        comb_list = rend_zip_df[rend_zip_df['prescribing_provider_npi'] == tin].values
    
        dfTmp = pd.DataFrame(random.choices(comb_list,
                                            k=random.choices([1, 2, 3,4,5], k=1)[0]),
                             columns = rend_zip_df.columns)
        i+=1
        dfTmp["ren_id"] = str(i)
        dfRendChoices = dfRendChoices.append(dfTmp)
    print("&"+str(datetime.datetime.now()))
    
    dfRendChoices['prescriber_zipcode'] = dfRendChoices['prescribing_provider_zipcode'].astype(str)
    dfRendChoices['patient_zipcode'] = dfRendChoices['patient_zipcode'].astype(str)
    dfRendChoices['pharmacy_zipcode']= dfRendChoices['filling_provider_zipcode'].astype(str)
    dfRendChoices = dfRendChoices.drop(columns = ['prescribing_provider_zipcode',"filling_provider_zipcode"])
    dfRendChoices['prescriber_city'] = dfRendChoices['prescriber_zipcode'].str[:5]
    dfRendChoices['patient_city'] = dfRendChoices['patient_zipcode'].str[:5]
    dfRendChoices['pharmacy_city'] = dfRendChoices['pharmacy_zipcode'].str[:5]
    
    dfRendChoices['pharmacy_city'] = dfRendChoices['pharmacy_city'].str.replace('.','')
    dfRendChoices['patient_city'] = dfRendChoices['patient_city'].str.replace('.','')
    dfRendChoices['prescriber_city'] = dfRendChoices['prescriber_city'].str.replace('.','')
    
    dfRendChoices['pharmacy_city'] = dfRendChoices['pharmacy_city'].astype(float)
    dfRendChoices['patient_city'] = dfRendChoices['patient_city'].astype(float)
    dfRendChoices['prescriber_city'] = dfRendChoices['prescriber_city'].astype(float)
    
    dfRendChoices['prescriber_city'] = dfRendChoices['prescriber_city'].fillna(0).astype(int).apply(zcode)
    dfRendChoices['patient_city'] = dfRendChoices['patient_city'].fillna(0).astype(int).apply(zcode)
    dfRendChoices['pharmacy_city'] = dfRendChoices['pharmacy_city'].fillna(0).astype(int).apply(zcode)
    dfRendChoices = dfRendChoices.rename(columns = {"filling_provider_state":"pharmacy_state", "prescribing_provider_state": "prescriber_state", "filling_provider_state":"pharmacy_state"})

    rend = dfRendChoices.values.tolist()
    out = []
    c =[]
    f= ""
    for a in rend:
        if f == a[14]:
            c.append(a)
        else:
            out.append(c)
            c = [a]
            f = (a[14]) 
    
    out.append(c)
    out.pop(0)
    final_df["Ren"] = final_df["ren_id"].apply(lambda x: random.choice(out[random.randint(0,len(out)-1)]))
    rend = (list(dfRendChoices.columns))
    del dfRendChoices
    num_lines= final_df.shape[0]

    final_df["Filling"] = np.random.choice(fill_lst, size = num_lines)
    spliting("Filling", "%")
    final_df[fill_col] = pd.DataFrame(final_df.pop('Filling').values.tolist(), index=final_df.index)
    #return final_df
    final_out = pd.DataFrame()
    state_lst = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS',
                 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV',
                 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
    # Split final DF based on age and gender so that a patient can be reasonably assigned to match the claim info.
    dfs = []
    for st in state_lst:
        print(st)
        final_df_st = final_df
        strOutFile = 's3://{}/{}'.format('synthdata-dev', "Patients/"+st+".csv")
        pat_df = pd.read_csv(strOutFile)
        pat_df["PatientID"] = pat_df["PatientID"].astype(str)
        pat_df["SubscriberID"] = pat_df["SubscriberID"].astype(str)
        pat_df["PatientZipCode"] = pat_df["PatientZipCode"].astype(str)
        pat_df["SubscriberZipCode"] = pat_df["SubscriberZipCode"].astype(str)
        pat_df["Pat"] = pat_df['PatientAddressLine1'] +'@'+pat_df['PatientAddressLine2'] +'@'+pat_df['PatientID'] +'@'+pat_df['PatientState'] +'@'+pat_df['PatientDOB'] +'@'+pat_df['PatientZipCode'] +'@'+pat_df['PatientGender'] +'@'+pat_df['PatientFName'] +'@'+pat_df['PatientMName'] +'@'+pat_df['PatientLName'] +'@'+pat_df['PatientRelationship'] +'@'+pat_df['age_bin']+'@'+pat_df['SubscriberAddressLine1'] +'@'+pat_df['SubscriberAddressLine2'] +'@'+pat_df['SubscriberID'] +'@'+pat_df['SubscriberState'] +'@'+pat_df['SubscriberDOB'] +'@'+pat_df['SubscriberZipCode'] +'@'+pat_df['SubscriberGender'] +'@'+pat_df['SubscriberFName'] +'@'+pat_df['SubscriberMName'] +'@'+pat_df['SubscriberLName']

        adult_df = pat_df.loc[(pat_df['age_bin'] == 'Adult')]
        adult_df_m = adult_df.loc[(adult_df['PatientGender'] == 'Male')]
        adult_df_f = adult_df.loc[(adult_df['PatientGender'] == 'Female')]
        adult_df_o = adult_df.loc[(adult_df['PatientGender'] == 'Other')]

        elderly_df = pat_df.loc[(pat_df['age_bin'] == 'Elderly')]    
        elderly_df_m = elderly_df.loc[(elderly_df['PatientGender'] == 'Male')]
        elderly_df_f = elderly_df.loc[(elderly_df['PatientGender'] == 'Female')]
        elderly_df_o = elderly_df.loc[(elderly_df['PatientGender'] == 'Other')]

        minor_df = pat_df.loc[(pat_df['age_bin'] == 'Minor')]
        minor_df_m = minor_df.loc[(minor_df['PatientGender'] == 'Male')]
        minor_df_f = minor_df.loc[(minor_df['PatientGender'] == 'Female')]
        minor_df_o = minor_df.loc[(minor_df['PatientGender'] == 'Other')]

        adult_list_m = adult_df_m["Pat"].tolist()
        adult_list_f = adult_df_f["Pat"].tolist()
        adult_list_o = adult_df_o["Pat"].tolist()

        elderly_list_m = elderly_df_m["Pat"].tolist()
        elderly_list_f = elderly_df_f["Pat"].tolist()
        elderly_list_o = elderly_df_o["Pat"].tolist()

        minor_list_m = minor_df_m["Pat"].tolist()
        minor_list_f = minor_df_f["Pat"].tolist()
        minor_list_o = minor_df_o["Pat"].tolist()
        
        final_df_st = final_df.loc[(final_df['pharmacy_state'] == st)]
        if st == "GA":
            final_df_null = final_df.loc[(final_df['pharmacy_state'] == "0.0")]
            final_df_st = final_df_st.append(final_df_null)
        final_df_st = final_df_st.reset_index(drop = True)
        if final_df_st.shape[0]==0:
            continue
        print(st)
        adult_df = final_df_st[(final_df_st['age_bin'] == 'Adult')]
        adult_df_m = adult_df[(adult_df['pat_gender'] == 'M')]
        adult_df_f = adult_df.loc[(adult_df['pat_gender'] == 'F')]
        adult_df_o = adult_df.loc[(adult_df['pat_gender'] == 'None')]
        
        adult_df_m["Pat"] = np.random.choice(adult_list_m, size=adult_df_m.shape[0])
        adult_df_f["Pat"] = np.random.choice(adult_list_f, size=adult_df_f.shape[0])
        adult_df_o["Pat"] = np.random.choice(adult_list_o, size=adult_df_o.shape[0])
        
        elderly_df = final_df_st.loc[(final_df_st['age_bin'] == 'Elderly')]
        elderly_df_m = elderly_df.loc[(elderly_df['pat_gender'] == 'M')]
        elderly_df_f = elderly_df.loc[(elderly_df['pat_gender'] == 'F')]
        elderly_df_o = elderly_df.loc[(elderly_df['pat_gender'] == 'None')]
        
        elderly_df_m["Pat"] = np.random.choice(elderly_list_m, size=elderly_df_m.shape[0])
        elderly_df_f["Pat"] = np.random.choice(elderly_list_f, size=elderly_df_f.shape[0])
        elderly_df_o["Pat"] = np.random.choice(elderly_list_o, size=elderly_df_o.shape[0])
        
        minor_df = final_df_st.loc[(final_df_st['age_bin'] == 'Minor')]
        minor_df_m = minor_df.loc[(minor_df['pat_gender'] == 'M')]
        minor_df_f = minor_df.loc[(minor_df['pat_gender'] == 'F')]
        minor_df_o = minor_df.loc[(minor_df['pat_gender'] == 'None')]
        
        minor_df_m["Pat"] = np.random.choice(minor_list_m, size=minor_df_m.shape[0])
        minor_df_f["Pat"] = np.random.choice(minor_list_f, size=minor_df_f.shape[0])
        minor_df_o["Pat"] = np.random.choice(minor_list_o, size=minor_df_o.shape[0])
        final_df_st = pd.concat([minor_df_m, minor_df_f, minor_df_o, elderly_df_m, elderly_df_f, elderly_df_o,
                                 adult_df_f,adult_df_m, adult_df_o])
        del adult_df
        del elderly_df
        del minor_df
        del adult_df_m
        del adult_df_f
        del adult_df_o
        del elderly_df_m
        del elderly_df_f
        del elderly_df_o
        del minor_df_m
        del minor_df_f
        del minor_df_o
        #final_df_st=final_df_st.reset_index()
        final_out = final_out.append(final_df_st)
        #dfs.append(final_df_st)
        del final_df_st
    
    #final_df = pd.concat(dfs, ignore_index=True)
    final_df = final_out
    
    final_df["written_date"] = np.random.choice(pd.date_range('2020-01-01', '2022-01-01'), len(final_df))
    delta = pd.to_timedelta(np.random.randint(0,4, size=len(final_df)), unit='d')
    final_df["SubmitDate"] = final_df["written_date"]+delta
    delta = pd.to_timedelta(np.random.randint(0,3, size=len(final_df)), unit='d')
    final_df["processed_date"] = final_df["SubmitDate"] +delta
    delta = pd.to_timedelta(np.random.randint(0,3, size=len(final_df)), unit='d')
    final_df["received_date"] = final_df["processed_date"]+delta
    delta = pd.to_timedelta(np.random.randint(0,5, size=len(final_df)), unit='d')
    final_df["paid_date"]=final_df["received_date"]+delta 
    delta = pd.to_timedelta(np.random.randint(0,3, size=len(final_df)), unit='d')
    final_df["final_claim_date"] = final_df["paid_date"]+delta
    print("@"+str(datetime.datetime.now()))
    spliting("Pat")


    pat_lst = ['patient_address_line1', 'patient_address_line2', 'patient_id', 'PatientState', 'PatientDOB', 
               'PatientZipCode', 'PatientGender', 'patient_first_name', 'patient_middle_name', 'patient_last_name',
               'patient_relationship', 'age_bin','SubscriberAddressLine1', 'SubscriberAddressLine2', 'sub_id',
               'SubscriberState', 'SubscriberDOB', 'SubscriberZipCode', 'SubscriberGender', 'SubscriberFName', 
               'SubscriberMName', 'SubscriberLName']
    final_df[pat_lst] = pd.DataFrame(final_df.pop('Pat').values.tolist(), index=final_df.index)

    final_df[rend] = pd.DataFrame(final_df.pop('Ren').values.tolist(), index=final_df.index)
    
    nz_df= final_df.loc[(final_df['prescriber_zipcode'] == "")]
    z_df=final_df.loc[(final_df['prescriber_zipcode'] != "")]
    nz_df["pharmacy_address_line1"] =""
    nz_df["pharmacy_address_line2"] =""    
    final_df = pd.concat([nz_df, z_df])        
    e_df = final_df.loc[(final_df['age_bin'] == 'Elderly')]
    n_df = final_df.loc[(final_df['age_bin'] != 'Elderly')]    
    e_df["line_of_business"] = np.random.choice(["Medicare", "Commercial Plus"],  e_df.shape[0], p=[0.95, 0.05])
    n_df["line_of_business"] = np.random.choice(["Commercial", "Medicare", "Commercial Plus"], n_df.shape[0], p=[0.5, 0.05, 0.45])
    final_df = pd.concat([n_df, e_df])    
    del n_df
    del e_df
    final_df["claim_adjustment_code"]=""
    final_df["claim_adjustment_reason_code"]=""
    final_df["claim_denial_clarification_code"]=""
    final_df["claim_deletion_ind"]=""
    final_df["prior_claim_id"]=""
    final_df["specialty_ind"]=""
    final_df["generic_drug_indicator"]=""
    final_df["cob"] = 0
    final_df["other_drug_code"]=''
    final_df["mail_order_indicator"] = np.random.choice(["Y", "N"],  final_df.shape[0], p=[0.8, 0.2])
    final_df["drug_utilization_review_code"]=""
    final_df["SubmitTimestamp"] = [fake.time() for i in range(final_df.shape[0])]
    final_df["group_number"]=  np.random.randint(1000, 9999, size=final_df.shape[0]).astype(str)
    final_df["prescription_number"] = np.random.randint(10000000000,999999999999, size=final_df.shape[0]).astype(str)#needs to be the same across claims find
    final_df["claim_auth-number"] = np.random.randint(10000000,999999999, size=final_df.shape[0]).astype(str)
    final_df["rx_origin_code"]= np.random.choice(["phone", "fax", 'verbal', 'written'], final_df.shape[0], p=[0.2,0.6, 0.15, 0.05])
    final_df["claim_source_code"]=""
    final_df["formulary_indicator"] = np.random.choice(["Y", "N"],  final_df.shape[0], p=[0.6, 0.4])
    final_df['claim_ncpdp_number']  =  np.random.randint(1000000000, 9999999999, size=final_df.shape[0]).astype(str)
    final_df["affiliate_code"] =  np.random.randint(1000, 9999, size=final_df.shape[0]).astype(str)
    final_df["pharmacy_chain_code"]= np.random.randint(100, 999, size=final_df.shape[0]).astype(str)
    final_df["prescriber_specialty"] = str(taxonomy)
    combo_list = final_df["combos"].tolist()
    final_df['ren_id'] = final_df['ren_id'].astype(str)
    final_df = final_df.explode('combos')#spliting up line into HCPCS and mod value in list, first is Hcpcs, second is mod for line
    final_df["combos"] = final_df["combos"].apply(lambda x: x.split(";"))
    final_df[['ndccd','metric_quantity_dispensed']] = pd.DataFrame(final_df.pop('combos').values.tolist(), index=final_df.index)
    final_df = final_df.drop(columns=['Unnamed: 0',"zip"])
    
    strOutFile = 's3://{}/{}'.format('synthdata-dev/Pharmacy', "ndc_w_gpi_14.csv")#ndc_w_all_gpi.csv this is another file that has all ndc codes rather than just ndc 14 codes which are the current codes
    
    merge_df = pd.read_csv(strOutFile)
    merge_df["ndccd"]=merge_df["ndccd"].astype(str)
    
    merge_df = merge_df.sort_values(by=['generic_drug_name'], ascending=False)

    final_df["ndccd"] = final_df["ndccd"].astype(str)
    final_df['ndccd'] = final_df['ndccd'].str.lstrip('0')

    merge_df=merge_df.drop_duplicates(subset='ndccd')
    merge_df["ndccd"] = merge_df['ndccd'].str.lstrip('0')
    merge_df["ndccd"] = merge_df["ndccd"].astype('int64', copy=False, errors='ignore')
    merge_df['ndccd'] = merge_df['ndccd'].astype(str).str[:-2]
    final_df = final_df.merge(merge_df, how='left', on='ndccd')

    def nopay2(x):
        if x["ndccd"] in nopay_lst:
            return  np.random.choice([ "Rejected", "Additional Information Requested"], p=[0.7, 0.3,])
        return "Accepted"
    final_df['ndccd'] = final_df['ndccd'].apply(lambda x: '{0:0>11}'.format(x))
    final_df["claim_status_category_code"]=final_df.apply(nopay2, axis=1)
    final_df["Claim_status"] = final_df.apply(nopay2, axis=1)
    final_df["deny_flag"] = np.random.choice(["Y", "N"],  final_df.shape[0], p=[0.05, 0.95])
    final_df = final_df.drop(columns=[ "prob_size", "size"])
    final_df = final_df.rename(columns = {"druglabelname":"drug_label_name"})
    
    final_df["combo"] = final_df["ndccd"] +"@"+final_df["metric_quantity_dispensed"]
    print("#"+str(datetime.datetime.now()))
    final_df['PrescriptionNumber']= final_df["number_refills_auth"]+1
    final_df['PrescriptionNumber'] = final_df['PrescriptionNumber'].astype('int64', copy=False, errors='ignore')
    final_df['PrescriptionNumber'] = final_df.apply(lambda x: list(range(1, x.PrescriptionNumber + 1)), axis=1)
    

    
    final_df= final_df.explode('PrescriptionNumber')
    
    final_df = final_df.drop(columns=['PrescriptionNumber'])
    final_df = final_df.rename(columns = {"prescription_number":"PrescriptionNumber"})
    refill = pd.read_csv(refill_csv)
            
    c = pd.read_csv(c_amt_csv)
    d = pd.read_csv(d_amt_csv)
    
    ######
    scb = 1      #may want to change scale so that it is more in line with reality based on fraudscope returns
    sca = .05
    
    cost_dict = c.set_index('combo').to_dict()
    final_df['cost'] = final_df['combo'].apply(lambda x: cost_dict["avg_charge"][x])
    final_df['amount_charged'] = np.random.normal(final_df['cost'], scale = scb).astype(int)
    final_df['amount_charged'] = final_df['amount_charged'].abs()
    final_df = final_df.drop(columns=['cost'])
    
    paid_dict = d.set_index('combo').to_dict()
    final_df['paid'] = final_df['combo'].apply(lambda x: paid_dict["avg_paid_pct"][x])
    final_df['amount_paid'] = np.random.normal(final_df['paid'], scale = sca)*final_df["amount_charged"].astype(int)
    final_df['amount_paid'] = final_df['amount_paid'].abs()
    final_df = final_df.drop(columns=['paid'])
    final_df["amount_paid"] = final_df["amount_paid"].astype('int64', copy=False, errors='ignore')
    
    final_df = final_df.drop(columns=['age_bin',"ren_id",'combo','combo_tot'])
    final_df["dispensing_fee_payable"] = 1.15
    final_df["coins_amount"] = np.random.uniform(0, 5.42, size = final_df.shape[0])
    final_df.coins_amount = final_df.coins_amount.round(2)
    
    final_df["other_insurance_indicator"] =  np.random.choice(["N", "Y"],  final_df.shape[0], p=[0.95, 0.05])
    nm_df = final_df.loc[(final_df['amount_paid'] <= 20)]
    sm_df = final_df.loc[(final_df['amount_paid'] >= 20)]
    sm_df["flat_copay_amount_paid"] =  np.random.choice([5, 10, 15],  sm_df.shape[0], p=[0.4, 0.4, 0.2])
    sm_df["deductible_amount"] = (sm_df["amount_paid"]-sm_df["flat_copay_amount_paid"])*.73
    nm_df["flat_copay_amount_paid"] =  np.random.choice([5, 0],  nm_df.shape[0], p=[0.4, 0.6])
    nm_df["deductible_amount"] = nm_df["amount_paid"]-nm_df["flat_copay_amount_paid"]
    final_df = pd.concat([sm_df, nm_df])
    final_df.deductible_amount = final_df.deductible_amount.round(2)
    final_df['days_supply']=  np.random.choice([7, 30, 60,90],  final_df.shape[0], p=[0.2, 0.4, 0.2, 0.2])
    final_df["dispensing_fee_payable"]= 1.15
    final_df["amount_paid"] = final_df["amount_paid"]+final_df["coins_amount"]
    final_df["patient_pay_amount"] = final_df["flat_copay_amount_paid"]+ final_df["dispensing_fee_payable"]
    final_df["ingredient_cost_charged"] = final_df["amount_paid"]+final_df["patient_pay_amount"] -final_df["dispensing_fee_payable"]
    final_df["allowed_amount"] = final_df["ingredient_cost_charged"]*.92
    final_df.ingredient_cost_charged = final_df.ingredient_cost_charged.round(2)
    final_df["contract_fee"] =final_df["amount_paid"] - final_df["patient_pay_amount"]
    final_df["apprvd_ingred_cost"] = final_df["ingredient_cost_charged"]
    final_df["sales_tax_amount"] = final_df["patient_pay_amount"]*.04
    final_df.sales_tax_amount = final_df.sales_tax_amount.round(2)
    final_df['sales_tax_amount_aprroved'] = final_df["sales_tax_amount"]
    final_df["usual_and_customary_amount_sub"] = final_df["allowed_amount"]
    final_df.allowed_amount = final_df.allowed_amount.round(2)
    final_df.contract_fee = final_df.contract_fee.round(2)
    final_df['ndccd'] = final_df['ndccd'].apply(lambda x: '{0:0>11}'.format(x))
    final_df["prior_authorization_flag"] =final_df["ndccd"].apply(lambda x: preauth(x))
    final_df["AHFS_Code"]=""
    final_df['amount_paid'] = final_df['amount_paid'].abs()
    final_df = final_df.rename(columns = {"drug_route_values":"drug_route_code"})#, inplace=True)
    
    deny_df = final_df.loc[(final_df['deny_flag'] == 'Y')]
    nd_df = final_df.loc[(final_df['deny_flag'] == 'N')]
    deny_df["amount_paid"] = 0
    deny_df["prior_authorization_flag"] = 0
    deny_df["sales_tax_amount_aprroved"] = 0
    deny_df["apprvd_ingred_cost"] = 0
    deny_df["contract_fee"] = 0
    deny_df["patient_pay_amount"] = 0
    deny_df["deductible_amount"] = 0
    
    
    
    final_df = pd.concat([deny_df, nd_df])
    final_df['custclaim_id'] = range(1, 1+len(final_df))
    final_df['custclaim_id']=final_df['custclaim_id'].astype(str)
    final_df["1"] = str(taxonomy)
    final_df["1"]=final_df["1"].apply(lambda x: '{0:0>3}'.format(x))
    final_df['custclaim_id'] = "2" + final_df['1']+final_df['custclaim_id']
    final_df = final_df.drop(columns=['1'])#custclaim_id
    #final_df = final_df.drop(columns=['deny_flag'])
    final_df["prior_authorization_flag"] = final_df["prior_authorization_flag"].astype(str)
    final_df["prior_authorization_flag"] = final_df["prior_authorization_flag"].replace({"0": "N", "1": "Y"})
    
    final_df["compound_flag"] = final_df["compound_flag"].astype(str)
    final_df["compound_flag"] = final_df["compound_flag"].replace({"0": "N", "1": "Y"})
    
    final_df["daw"] = final_df["daw"].astype(str)
    final_df["daw"] = final_df["daw"].replace({"N": "0", "Y": "3"})
    
    final_df["package_quantity"] = final_df["package_quantity"].astype('int64', copy=False, errors='ignore')
    final_df["dosage_strength"] = final_df["dosage_strength"].astype('int64', copy=False, errors='ignore')
    final_df["therapeutic_class"] =final_df["therapeutic_class"].astype('int64', copy=False, errors='ignore')
    final_df["deaclass"] = final_df["deaclass"].astype('int64', copy=False, errors='ignore')
    final_df["ndcgpi"] = final_df["ndcgpi"].astype('int64', copy=False, errors='ignore')
    
    final_df = final_df.drop_duplicates(subset = ['sub_id', 'custclaim_id'],keep = 'last').reset_index(drop = True)
    final_df=final_df.replace([np.inf, -np.inf], np.nan, inplace=False)
    final_df = final_df[["custclaim_id","prescription_number","prescriber_dea_number","prior_authorization_flag","ndccd","drug_label_name","prescriptiondays_count","package_quantity","dosage_strength","dosage_unit","number_refills_auth","metric_quantity_dispensed","filling_provider_npi","filling_provider_fname", "filling_provider_lname", "filling_provider_custid","filling_provider_taxid","pharmacy_address_line1","pharmacy_address_line2","pharmacy_city","pharmacy_state","pharmacy_zipcode","pharmacy_phone_number","mail_order_indicator","prescribing_provider_npi","prescribing_provider_name","prescribing_provider_custid","prescribing_provider_taxid","prescriber_address_line1","prescriber_address_line2","prescriber_city","prescriber_state","prescriber_zipcode","prescriber_phone_number","patient_id","patient_first_name","patient_middle_name","patient_last_name","patient_relationship","pat_gender","PatientDOB","patient_address_line1","patient_address_line2","patient_city","patient_state","patient_zipcode","sub_id","line_of_business","group_number","days_supply","therapeutic_class","deaclass","ndcgpi","compound_flag","dispensing_fee_payable","amount_charged","ingredient_cost_charged","amount_paid","patient_pay_amount","flat_copay_amount_paid","coins_amount","contract_fee","carrier_id","formulary_indicator","written_date","processed_date","prescriber_specialty","filler_specialty","otc_ind","daw","refill_code","generic_drug_name","cob","Claim_status","received_date","paid_date","final_claim_date","dispenser_type_code","other_drug_code","gpi_drug_group","gpi_drug_class","gpi_drug_subclass","gpi_drug_name","gpi_generic_drug_name","drug_compound_code","maintenance_drug_code","multisource_code","drug_route_code","claim_ncpdp_number","drug_utilization_review_code","claim_denial_clarification_code","other_insurance_indicator","allowed_amount","deductible_amount","sales_tax_amount","sales_tax_amount_aprroved","usual_and_customary_amount_sub","claim_source_code","pharmacy_nabp","dispenser_type","generic_drug_indicator","pharmacy_chain_code","pharmacy_affiliation","rx_origin_code","claim_auth-number","SubmitDate","SubmitTimestamp","AHFS_Code","apprvd_ingred_cost","specialty_ind","route_of_admin","claim_status_category_code","claim_adjustment_code","claim_adjustment_reason_code","prior_claim_id","claim_deletion_ind","affiliate_code","network_number"]]
    final_df.to_csv("Out/Pharmacy/"+str(folder)+".csv",  index=False)#sep="|",
    return final_df
    

In [7]:
import subprocess
subprocess.call(['bash', 'Test.sh'])


127

In [8]:
import subprocess
subprocess.call(['bash', 'Test.sh'])
"""Taxonomy, folder_name, amount of providers(lower => more)"""
print("0")
#fin = syn_data(6, 'Anesthesiology',50)
#print("1")
syn_data(22, 'Counseling_Psychology',15)
#print("2")
#syn_data(31, 'Emergency_Medicine',15)
#print("3")
#syn_data(66,'Nuclear_Radiology',15)
#print("4")
#syn_data(75, 'Orthopaedic_Surgery',15)
#print("5")
#syn_data(70, 'Obstetrics_Gynecology',15)
#print("6")
#syn_data(72, 'Ophthalmology',15)
#print("7")
#syn_data(84, 'Dermatology',15)
#print("9")
#fin = syn_data(108, 'Pediatrics',15)
#print("10")
#print("8")
##syn_data(121, 'Psychiatry_Neurology',15)
#
fin


0
!2022-05-16 17:43:21.572582
58
^2022-05-16 17:43:21.955437
0
10
20
30
40
50
&2022-05-16 17:43:22.372621
AK
AK
AL
AL
AR
AR
AZ
AZ
CA
CA
CO
CO
CT
CT
DC
DC
DE
DE
FL
FL
GA
GA
HI
HI
IA
IA
ID
ID
IL
IL
IN
IN
KS
KS
KY
KY
LA
LA
MA
MA
MD
MD
ME
ME
MI
MI
MN
MN
MO
MO
MS
MS
MT
MT
NC
NC
ND
ND
NE
NE
NH
NH
NJ
NJ
NM
NM
NV
NV
NY
NY
OH
OH
OK
OK
OR
OR
PA
PA
RI
RI
SC
SC
SD
SD
TN
TN
TX
TX
UT
UT
VA
VA
VT
VT
WA
WA
WI
WI
WV
WV
WY
WY
@2022-05-16 17:43:44.297666
#2022-05-16 17:43:45.387530


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
pd.set_option('display.max_columns', None)


In [ ]:
fin["PrescriptionNumber"]

In [ ]:
fin["custclaim_id"]
#dfProfData[dfProfData['RenderingProviderEntityType'] == 'O']

In [ ]:
fin[fin["PrescriptionNumber"] == 3]


In [ ]:
fin[fin["Claim_status"]=="Rejected"]

In [ ]:
fin["prescriber_specialty"]

In [ ]:
fail

In [ ]:
all_filenames = ["Out/Pharmacy/Anesthesiology.csv", "Out/Pharmacy/Counseling_Psychology.csv", "Out/Pharmacy/Emergency_Medicine.csv", "Out/Pharmacy/Nuclear_Radiology.csv", "Out/Pharmacy/Obstetrics_Gynecology.csv", "Out/Pharmacy/Ophthalmology.csv", "Out/Pharmacy/Orthopaedic_Surgery.csv", "Out/Pharmacy/Dermatology.csv"]
final_df = pd.concat((pd.read_csv(f, header = 0) for f in all_filenames))
print("@")
#final_df = final_df.drop_duplicates(subset = ['custclaim_id'],keep = 'last').reset_index(drop = True)
final_df["filling_provider_name"] = final_df["filling_provider_fname"]+" "+final_df["filling_provider_lname"]
final_df = final_df.drop(columns=["filling_provider_fname","filling_provider_lname","deny_flag","pharmacy_dba_name", "doctor_name", "pharmacy_phone_number_extension", "pharmacy_fax", "pharmacy_email", "pharmacy_cross_street_or_directions", "pharmacy_county", "pharmacy_msa", "pharmacy_pmsa", "pharmacy_24_hour_operation", "pharmacy_provider_hours", "pharmacy_congressional_voting_district", "pharmacy_language_code_1", "pharmacy_language_code_2", "pharmacy_language_code_3", "pharmacy_language_code_4", "pharmacy_language_code_5", "pharmacy_store_open_date", "pharmacy_store_closure_date", "pharmacy_mailing_address1", "pharmacy_mailing_address2", "pharmacy_mailing_address_city", "pharmacy_mailing_address_state", "pharmacy_mailing_address_zip", "pharmacy_contact_middle_initial", "pharmacy_contact_title", "pharmacy_contact_phone", "pharmacy_contact_extension", "pharmacy_contact_email", "filling_secondary_provider_type_code", "filling_tertiary_provider_type_code", "filling_dea_registration_number", "filling_dea_expiration_date", "filling_state_income_tax_id", "filling_deactivation_code", "filling_reinstatement_code", "filling_reinstatement_date", "drug_compound_desc", "maintenance_drug_desc", "multisource_desc"])
final_df.to_csv("Out/To_s3/.csv", sep="|", index=False)
final_df = final_df.sort_values(by=['sub_id'])
final_df = final_df.sort_values(by=['written_date'])
final_df["Date"] = final_df["written_date"]
final_df["Date"] = final_df["Date"].astype(str)
final_df["Date"] = final_df["Date"].str[:7]
final_df = final_df.drop_duplicates()
strBucket = 'synthdata-dev'
out = final_df
print("!")
out = out.drop(columns=['Date'])
i = 0
j = 0
k = 0
c = ""
for index, row in final_df.iterrows():
    j+=1
    if not (c ==row["Date"]):
        c = row["Date"]
        if k!=0:
            strBucket = 'synthdata-dev'
            strTestFile = "Pharmacy/Month_"+str(i)+'.csv'
            strOutFile = 's3://{}/{}'.format(strBucket, strTestFile)
            out.iloc[k:j-1].to_csv(strOutFile, sep="|", index=False)
        k = j
        i+=1
fold_lst1=["Out/Pharmacy/Anesthesiologyprov_file.csv", "Out/Pharmacy/Counseling_Psychologyprov_file.csv", "Out/Pharmacy/Emergency_Medicineprov_file.csv", "Out/Pharmacy/Nuclear_Radiologyprov_file.csv", "Out/Pharmacy/Obstetrics_Gynecologyprov_file.csv", "Out/Pharmacy/Ophthalmologyprov_file.csv", "Out/Pharmacy/Orthopaedic_Surgeryprov_file.csv", "Out/Pharmacy/Psychiatry_Neurologyprov_file.csv", "Out/Pharmacy/Pediatricsprov_file.csv", "Out/Pharmacy/Dermatologyprov_file.csv"]
fold_lst2=["Out/Pharmacy/Anesthesiologyrend_file.csv", "Out/Pharmacy/Counseling_Psychologyrend_file.csv", "Out/Pharmacy/Emergency_Medicinerend_file.csv", "Out/Pharmacy/Nuclear_Radiologyrend_file.csv", "Out/Pharmacy/Obstetrics_Gynecologyrend_file.csv", "Out/Pharmacy/Ophthalmologyrend_file.csv", "Out/Pharmacy/Orthopaedic_Surgeryrend_file.csv", "Out/Pharmacy/Psychiatry_Neurologyrend_file.csv", "Out/Pharmacy/Pediatricsrend_file.csv", "Out/Pharmacy/Dermatologyrend_file.csv"]
# file 1
combined_csv = pd.concat([pd.read_csv(f, header=0) for f in fold_lst1 ])
combined_csv = combined_csv.drop_duplicates()
strOutFile = 's3://{}/{}'.format('synthdata-dev', "Pharmacy/File1.csv")
combined_csv.to_csv(strOutFile, sep="|", index=False)
#file 2
combined_csv = pd.concat([pd.read_csv(f, header=0) for f in fold_lst2 ])
combined_csv = combined_csv.drop_duplicates()
strOutFile = 's3://{}/{}'.format('synthdata-dev', "Pharmacy/File2.csv")
combined_csv.to_csv(strOutFile, sep="|", index=False)


In [ ]:
din = fin.loc[fin['number_refills_auth'] == 11.0]
din["number_refills_auth"]

In [ ]:

#final_df["patient_pay_amount"]=final_df["deductible_amount"]

In [ ]:
col = list(fin.columns)
correct = ["custclaim_id", "prescription_number", "prescriber_dea_number", "prior_authorization_flag", "ndccd", "drug_label_name", "prescriptiondays_count", "package_quantity", "dosage_strength", "dosage_unit", "number_refills_auth", "metric_quantity_dispensed", "filling_provider_npi", "filling_provider_fname","filling_provider_lname", "filling_provider_custid", "filling_provider_taxid", "pharmacy_address_line1", "pharmacy_address_line2", "pharmacy_city", "pharmacy_state", "pharmacy_zipcode", "pharmacy_phone_number", "mail_order_indicator", "prescribing_provider_npi", "prescribing_provider_name", "prescribing_provider_custid", "prescribing_provider_taxid", "prescriber_address_line1", "prescriber_address_line2", "prescriber_city", "prescriber_state", "prescriber_zipcode", "prescriber_phone_number", "patient_id", "patient_first_name", "patient_middle_name", "patient_last_name", "patient_relationship", "pat_gender", "birth_date", "patient_address_line1", "patient_address_line2", "patient_city", "patient_state", "patient_zipcode", "sub_id", "line_of_business", "group_number", "days_supply", "therapeutic_class", "deaclass", "ndcgpi", "compound_flag", "dispensing_fee_payable", "amount_charged", "ingredient_cost_charged", "amount_paid", "patient_pay_amount", "flat_copay_amount_paid", "coins_amount", "contract_fee", "carrier_id", "formulary_indicator", "written_date", "processed_date", "prescriber_specialty", "filler_specialty", "otc_ind", "daw", "refill_code", "generic_drug_name", "cob", "Claim_status", "received_date", "paid_date", "final_claim_date", "dispenser_type_code", "other_drug_code", "gpi_drug_group", "gpi_drug_class", "gpi_drug_subclass", "gpi_drug_name", "gpi_generic_drug_name", "drug_compound_code", "maintenance_drug_code", "multisource_code", "drug_route_code", "claim_ncpdp_number", "drug_utilization_review_code", "claim_denial_clarification_code", "other_insurance_indicator", "allowed_amount", "deductible_amount", "sales_tax_amount", "sales_tax_amount_aprroved", "usual_and_customary_amount_sub", "claim_source_code", "pharmacy_nabp", "dispenser_type", "generic_drug_indicator", "pharmacy_chain_code", "pharmacy_affiliation", "rx_origin_code", "claim_auth-number", "SubmitDate", "SubmitTimestamp", "AHFS_Code", "apprvd_ingred_cost", "specialty_ind", "route_of_admin", "claim_status_category_code", "claim_adjustment_code", "claim_adjustment_reason_code", "prior_claim_id", "claim_deletion_ind", "affiliate_code", "network_number"]
for a in correct:
    if not a in col:
        print(a)
for a in col:
    if not a in correct:
        print(a)

In [ ]:
col = list(final_df.columns)
correct = ["custclaim_id", "prescription_number", "prescriber_dea_number", "prior_authorization_flag", "ndccd", "drug_label_name", "prescriptiondays_count", "package_quantity", "dosage_strength", "dosage_unit", "number_refills_auth", "metric_quantity_dispensed", "filling_provider_npi", "filling_provider_fname","filling_provider_lname", "filling_provider_custid", "filling_provider_taxid", "pharmacy_address_line1", "pharmacy_address_line2", "pharmacy_city", "pharmacy_state", "pharmacy_zipcode", "pharmacy_phone_number", "mail_order_indicator", "prescribing_provider_npi", "prescribing_provider_name", "prescribing_provider_custid", "prescribing_provider_taxid", "prescriber_address_line1", "prescriber_address_line2", "prescriber_city", "prescriber_state", "prescriber_zipcode", "prescriber_phone_number", "patient_id", "patient_first_name", "patient_middle_name", "patient_last_name", "patient_relationship", "pat_gender", "birth_date", "patient_address_line1", "patient_address_line2", "patient_city", "patient_state", "patient_zipcode", "sub_id", "line_of_business", "group_number", "days_supply", "therapeutic_class", "deaclass", "ndcgpi", "compound_flag", "dispensing_fee_payable", "amount_charged", "ingredient_cost_charged", "amount_paid", "patient_pay_amount", "flat_copay_amount_paid", "coins_amount", "contract_fee", "carrier_id", "formulary_indicator", "written_date", "processed_date", "prescriber_specialty", "filler_specialty", "otc_ind", "daw", "refill_code", "generic_drug_name", "cob", "Claim_status", "received_date", "paid_date", "final_claim_date", "dispenser_type_code", "other_drug_code", "gpi_drug_group", "gpi_drug_class", "gpi_drug_subclass", "gpi_drug_name", "gpi_generic_drug_name", "drug_compound_code", "maintenance_drug_code", "multisource_code", "drug_route_code", "claim_ncpdp_number", "drug_utilization_review_code", "claim_denial_clarification_code", "other_insurance_indicator", "allowed_amount", "deductible_amount", "sales_tax_amount", "sales_tax_amount_aprroved", "usual_and_customary_amount_sub", "claim_source_code", "pharmacy_nabp", "dispenser_type", "generic_drug_indicator", "pharmacy_chain_code", "pharmacy_affiliation", "rx_origin_code", "claim_auth-number", "SubmitDate", "SubmitTimestamp", "AHFS_Code", "apprvd_ingred_cost", "specialty_ind", "route_of_admin", "claim_status_category_code", "claim_adjustment_code", "claim_adjustment_reason_code", "prior_claim_id", "claim_deletion_ind", "affiliate_code", "network_number"]
for a in correct:
    if not a in col:
        print(a)
print("_________")
for a in col:
    if not a in correct:
        print(a)

In [ ]:
col = ["nabp_number", "legal_business_name", "dba_name", "doctor_name", "store_number", "address1", "address2", "city", "state", "phone", "extension", "fax", "email", "cross_street_or_directions", "county", "msa", "pmsa", "_24_hour_operation", "provider_hours", "congressional_voting_district", "language_code_1", "language_code_2", "language_code_3", "language_code_4", "language_code_5", "store_open_date", "store_closure_date", "mailing_address1", "mailing_address2", "mailing_address_city", "mailing_address_state", "mailing_address_zip", "contact_last_name", "contact_first_name", "contact_middle_initial", "contact_title", "contact_phone", "contact_extension", "contact_email", "dispenser_class_code", "primary_provider_type_code", "secondary_provider_type_code", "tertiary_provider_type_code", "medicare_provider_id", "npi", "dea_registration_number", "dea_expiration_date", "tax_id", "state_income_tax_id", "deactivation_code", "reinstatement_code", "reinstatement_date"]
print(len(col))



In [ ]:
pool = multiprocessing.Pool(args.nprocs)
for i in range(args.min_partition, args.max_partition + 1):
    pool.apply_async(run, (copy.deepcopy(options), ),
    callback=cb_ok, error_callback=cb_error)
    poolResDic[i] = poolRes
    

In [ ]:
fold_lst1=["Out/Pharmacy/Anesthesiologyprov_file.csv", "Out/Pharmacy/Counseling_Psychologyprov_file.csv", "Out/Pharmacy/Emergency_Medicineprov_file.csv", "Out/Pharmacy/Nuclear_Radiologyprov_file.csv", "Out/Pharmacy/Obstetrics_Gynecologyprov_file.csv", "Out/Pharmacy/Ophthalmologyprov_file.csv", "Out/Pharmacy/Orthopaedic_Surgeryprov_file.csv", "Out/Pharmacy/Psychiatry_Neurologyprov_file.csv", "Out/Pharmacy/Pediatricsprov_file.csv"]
fold_lst2=["Out/Pharmacy/Anesthesiologyrend_file.csv", "Out/Pharmacy/Counseling_Psychologyrend_file.csv", "Out/Pharmacy/Emergency_Medicinerend_file.csv", "Out/Pharmacy/Nuclear_Radiologyrend_file.csv", "Out/Pharmacy/Obstetrics_Gynecologyrend_file.csv", "Out/Pharmacy/Ophthalmologyrend_file.csv", "Out/Pharmacy/Orthopaedic_Surgeryrend_file.csv", "Out/Pharmacy/Psychiatry_Neurologyrend_file.csv", "Out/Pharmacy/Pediatricsrend_file.csv"]
combined_csv = pd.concat([pd.read_csv(f, header=0) for f in fold_lst1 ])
combined_csv = combined_csv.drop_duplicates()
combined_csv



In [ ]:
strBucket = 'synthdata-dev'
strTestFile = "Pharmacy/Month_7.csv"
strOutFile = 's3://{}/{}'.format(strBucket, strTestFile)
df= pd.read_csv(strOutFile, sep="|")
            
df        
            
            

In [ ]:
strOutFile = 's3://{}/{}'.format('synthdata-dev/Pharmacy', "ndc_w_gpi_14.csv")#ndc_w_all_gpi.csv
    #ndc_w_gpi_14.csv
    
merge_df = pd.read_csv(strOutFile)
merge_df

In [ ]:
strOutFile = 's3://{}/{}'.format('synthdata-dev/Pharmacy', "ndc_w_gpi_14.csv")#ndc_w_all_gpi.csv
    #ndc_w_gpi_14.csv
    
merge_df = pd.read_csv(strOutFile)
merge_df